# I. Set ups for Data
- Create data loader


In [1]:
import torch
from pathlib import Path

#device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"

#Hyper Parameters
RAND_SEED = 420
torch.manual_seed(RAND_SEED)

In [2]:
print(device)

cuda


## 1. Data Handling Process

### 1.1 Set up for data handling process

In [3]:
import h5py
#Data paths
data_folder_path = Path("dataset")
data_path = data_folder_path / "cylinder_flow"
train_path = data_path / "train"
test_path = data_path / "test"
dev_path = data_path / "valid"

CHANNELS =2 #(vx,vy)
GRID_SIZE =64

### 1.2 Datasets classes

In [4]:
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch

# check if its np arr
if not isinstance(sol_tensor,np.ndarray):
  print("Conversted sol solution to np array")
  sol_tensor = np.array(sol_tensor)

class BurgerDataset_Autoregressive_Sparse(Dataset):
    """
    A *faster* sliding window dataset (e.g., 10-in, 1-out).
    It creates a smaller, randomized dataset of (N_sims * windows_per_sim) samples.
    """
    def __init__(self, data_tensor, t_in=10, t_out=1, windows_per_sim=20):
        self.data = data_tensor # Shape: (Num_Sims, time_step, num_nodes, 2)
        self.t_in = t_in
        self.t_out = t_out
        self.total_steps = t_in + t_out 
        
        # Max possible start time
        self.max_start_time = self.data.shape[1] - self.total_steps 
        
        self.sims_in_dataset = self.data.shape[0]
        self.windows_per_sim = windows_per_sim
        
        # Total length of the dataset is now much smaller
        self.length = self.sims_in_dataset * self.windows_per_sim
        # e.g., 1000 sims * 20 windows/sim = 20,000 samples

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # 1. Figure out which simulation this index belongs to
        sim_index = idx // self.windows_per_sim
        
        # 2. Pick a RANDOM start time for the window
        # This makes the dataset different every epoch, which is good for training
        start_time_index = np.random.randint(0, self.max_start_time + 1)
        
        # 3. Define the end of the input and output slices
        end_in_index = start_time_index + self.t_in   # e.g., 50 + 10 = 60
        end_out_index = end_in_index + self.t_out    # e.g., 60 + 1 = 61

        # 4. Get the data chunks
        x_np = self.data[sim_index, start_time_index : end_in_index, :,:].astype(np.float32) # Shape (t_in, num_nodes,2 )
        y_np = self.data[sim_index, end_in_index : end_out_index, :,:].astype(np.float32)   # Shape (t_out,num_node,2)
        
        # 5. Transpose to (Points, Channels/Time) format
        x = x_np.transpose(1, 0) # Shape (1024, 10)
        y = y_np.transpose(1, 0) # Shape (1024, 1)
        
        return torch.from_numpy(x), torch.from_numpy(y)

print("BurgerDataset_Autoregressive_Sparse (FASTER dataset) defined.")

NameError: name 'sol_tensor' is not defined

In [5]:
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch
import os

class CylinderFlow_Autoregressive_Sparse(Dataset):
    """
    A *faster* sliding window dataset (e.g., 10-in, 1-out).
    It creates a smaller, randomized dataset of (N_sims * windows_per_sim) samples.
    """
    def __init__(self, data_dir, t_in=10, t_out=1, windows_per_sim=20):
        self.t_in = t_in
        self.t_out = t_out
        self.total_steps = t_in + t_out 
        self.data_dir = data_dir 
        self.file_paths = [os.path.join(self.data_dir,f) for f in os.listdir(self.data_dir)]
        self.example = torch.load(self.file_paths[0])

        
        # Max possible start time
        self.max_start_time = self.example["velocity"].shape[0]- self.total_steps
        self.sims_in_dataset = len(self.file_paths)
        self.windows_per_sim = windows_per_sim

        
        # Total length of the dataset is now much smaller
        self.length = self.sims_in_dataset * self.windows_per_sim
        # e.g., 1000 sims * 20 windows/sim = 20,000 samples

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # 1. Figure out which simulation this index belongs to
        sim_idx = idx // self.windows_per_sim
        file_path = self.file_paths[sim_idx]
        data = torch.load(file_path)
        velocity_traj = data["velocity"]
        
        # 2. Pick a RANDOM start time for the window
        # This makes the dataset different every epoch, which is good for training
        start_time_index = np.random.randint(0, self.max_start_time + 1)
        
        # 3. Define the end of the input and output slices
        end_in_index = start_time_index + self.t_in   # e.g., 50 + 10 = 60
        end_out_index = end_in_index + self.t_out    # e.g., 60 + 1 = 61

        # 4. Get the data chunks
        x_np = velocity_traj[start_time_index : end_in_index].to(torch.float32) # Shape (t_in, num_nodes,2 )
        y_np = velocity_traj[end_in_index : end_out_index].to(torch.float32)   # Shape (t_out,num_node,2)
        
        # Input: (10, 64, 64, 2) -> (64, 64, 10, 2) -> (64, 64, 20)
        x_tensor = x_np.permute(1, 2, 0, 3) 
        x_tensor = x_tensor.reshape(GRID_SIZE, GRID_SIZE, self.t_in * CHANNELS) 
        
        # Output: (5, 64, 64, 2) -> (64, 64, 5, 2) -> (64, 64, 10)
        y_tensor = y_np.permute(1, 2, 0, 3)
        y_tensor = y_tensor.reshape(GRID_SIZE, GRID_SIZE, self.t_out * CHANNELS)
        
        return x_tensor, y_tensor

print("BurgerDataset_Autoregressive_Sparse (FASTER dataset) defined.")

BurgerDataset_Autoregressive_Sparse (FASTER dataset) defined.


# II. Set up for model
- Declare up model class
- Set up loss functions
- Train - test function


## 1.Models set ups

### 1.1 FNO2D layer class

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
class FNO_Layer2D(nn.Module):
  """ A class act as one Fourier Layer as described in the original paper"""
  def __init__(self,in_chanels, out_chanels,mode_x,mode_y):
    super(FNO_Layer2D, self).__init__()
    self.in_chanels = in_chanels
    self.out_chanels = out_chanels
    self.mode_x = mode_x
    self.mode_y = mode_y
    #Scaler to scale the parameters
    self.scalar = (1/(in_chanels * out_chanels))
    self.weight_x = nn.Parameter(
        self.scalar * torch.rand(in_chanels,out_chanels,mode_x, mode_y, dtype = torch.cfloat)
    )
    self.weight_y = nn.Parameter(
        self.scalar * torch.rand(in_chanels,out_chanels,mode_x, mode_y,dtype = torch.cfloat)
    )

  def forward(self, x):
    #Size 
    batch,_,H,W = x.shape
      
    # FFT (2D fourier transform)
    x_ft = torch.fft.rfft2(x)
      
    ## Fourier layer
    out_ft = torch.zeros(
        batch, self.out_chanels,H, W //2 +1,
        dtype = torch.cfloat,
        device = device
    )

    out_ft[:,:,:self.mode_x,:self.mode_y] = torch.einsum(
        "bixy,ioxy->boxy",
        x_ft[:,:,:self.mode_x,:self.mode_y],
        self.weight_x
    )

    out_ft[:,:,-self.mode_x:,:self.mode_y] = torch.einsum(
        "bixy,ioxy->boxy",
        x_ft[:,:,-self.mode_x:, :self.mode_y],
        self.weight_y
    )
    ## Inverse FFT
    x = torch.fft.irfft2(out_ft , s=(H,W))
    return x
print("FNO_Layer2D class defined.")

FNO_Layer2D class defined.


### 1.2 FNO2D class

In [7]:
class FNO2D(nn.Module):
  def __init__(self ,width, mode_x, mode_y,in_chanels =1, out_chanels =200) :
    """ Default is 1 time step -> 200 timestep rest"""
    super(FNO2D,self).__init__()
    self.in_chanels = in_chanels + 2 # add one more for the grid
    self.out_chanels = out_chanels
    self.width = width
    self.mode_x = mode_x
    self.mode_y = mode_y
      
    # P layer ( lift the input chanel up):
    self.fc0 = nn.Linear(self.in_chanels, self.width)

    # T block: 4 Fourier layers
    self.conv0 = FNO_Layer2D(width,width,mode_x,mode_y)
    self.w0 = nn.Conv2d(self.width,self.width,1)

    self.conv1 = FNO_Layer2D(width,width,mode_x,mode_y)
    self.w1 = nn.Conv2d(self.width,self.width,1)

    self.conv2 = FNO_Layer2D(width,width,mode_x,mode_y)
    self.w2 = nn.Conv2d(self.width,self.width,1)

    self.conv3 = FNO_Layer2D(width,width,mode_x,mode_y)
    self.w3 = nn.Conv2d(self.width,self.width,1)

    # Q layer (project down to output_chanels)
    self.fc1 = nn.Linear(width,width*2)
    self.fc2 = nn.Linear(width*2, self.out_chanels)

  def forward(self,x):
    #Get the grid information
    grid = get_grid(shape = x.shape)

    # concat to the input
    x = torch.cat((x,grid),dim=-1) # (batch,H,W,chanels)

    ##Through P layer:
    x = self.fc0(x)

    x = x.permute(0,3,1,2) #(batch, chanel , H , W)

    ##Through T
    #1
    x1 = self.conv0(x)
    x2 = self.w0(x)
    x = x1+x2
    x = F.gelu(x)

    #2
    x1 = self.conv1(x)
    x2 = self.w1(x)
    x = x1+x2
    x = F.gelu(x)

    #3
    x1 = self.conv2(x)
    x2 = self.w2(x)
    x = x1+x2
    x = F.gelu(x)

    #4
    x1 = self.conv3(x)
    x2 = self.w3(x)
    x = x1+x2

    #Switch back
    x = x.permute(0, 2,3, 1) # (batch,H,W,chanels)

    ## Through Q
    x = self.fc1(x)
    x = F.gelu(x)
    x = self.fc2(x)
    return x

def get_grid(shape): 
    batch, H,W = shape[0], shape[1],shape[2]

    grid_x = torch.tensor(np.linspace(-1,1,H), dtype= torch.float)
    grid_y = torch.tensor(np.linspace(-1,1,W), dtype = torch.float)
    
    grid_x = grid_x.reshape(1, H, 1, 1).repeat([batch, 1, W, 1])
    grid_y = grid_y.reshape(1, 1, W, 1).repeat([batch, H, 1, 1])
    grid = torch.cat((grid_x, grid_y), dim =-1).to(device) 
    return grid 

print("FNO2D class defined.")

FNO2D class defined.


## 2.Loss functions


In [8]:
class LpLoss(object):
    """
    A class to compute the relative L2 loss (data loss).
    Calculates: mean( ||y_pred - y_true|| / ||y_true|| )
    """
    def __init__(self, size_average=True, d=2, p=2):
        super(LpLoss, self).__init__()
        self.d = d
        self.p = p
        self.size_average = size_average

    def cal_data_loss(self, y_pred, y_label):
        """
        Calculates the relative L2 error.
        y_pred: (Batch, Space, Time)
        y_label: (Batch, Space, Time)
        """

        # torch.norm(..., dim=(1,2)) calculates the L2 norm (length)
        # of the error for each sample in the batch across both
        # time and space dimensions.
        # Shape: (Batch,)
        diff_norms = torch.norm(y_pred - y_label, self.p, dim=(1, 2,3))
        y_label_norms = torch.norm(y_label, self.p, dim=(1, 2,3))

        # Calculate the relative error for each sample
        # We add a small epsilon to avoid division by zero
        rel_error = diff_norms / (y_label_norms + 1e-7)

        if self.size_average:
            # Return the average relative error across the batch
            return torch.mean(rel_error)
        else:
            # Return the sum of relative errors
            return torch.sum(rel_error)

    def __call__(self, x, y):
        return self.cal_data_loss(x, y)

### 2.2 Burger loss fn

In [9]:
## PINO loss function
class BurgerLoss(nn.Module):
    def __init__(self, t_coord, x_coord, viscosity=0.01, size_average=True):
        super(BurgerLoss, self).__init__()

        self.t_coord = t_coord
        self.x_coord = x_coord
        self.size_average = size_average
        self.viscosity = viscosity # Store viscosity

        # Calculate time step (dt) and space step (dx)
        self.dt = (t_coord[1] - t_coord[0]).item()
        self.dx = (x_coord[1] - x_coord[0]).item()

        print(f"Physics Loss Initialized:")
        print(f"  dt = {self.dt:.4f}")
        print(f"  dx = {self.dx:.4f}")
        print(f"  viscosity (nu) = {self.viscosity}")

        self.data_loss_fn = LpLoss(self.size_average)

    def cal_residual_loss(self, u):
        """
        Calculates the physics residual of the Burgers' equation:
        f = u_t + u * u_x - nu * u_xx

        We use central differences to approximate the derivatives.
        u shape is (Batch, Space, Time), e.g., (32, 1024, 201)
        """

        # --- 1. Calculate du/dt (Time derivative) ---
        # Slicing on dim=2 (Time)
        # Formula: (f(t+1) - f(t-1)) / (2*dt)
        u_t = (u[:, :, 2:] - u[:, :, :-2]) / (2 * self.dt)

        # --- 2. Calculate du/dx (Space derivative) ---
        # Slicing on dim=1 (Space)
        # Formula: (f(x+1) - f(x-1)) / (2*dx)
        u_x = (u[:, 2:, :] - u[:, :-2, :]) / (2 * self.dx)

        # --- 3. Calculate d2u/dx2 (Second space derivative) ---
        # Slicing on dim=1 (Space)
        # Formula: (f(x+1) - 2*f(x) + f(x-1)) / (dx^2)
        u_xx = (u[:, 2:, :] - 2 * u[:, 1:-1, :] + u[:, :-2, :]) / (self.dx**2)

        # --- 4. Align Tensors ---
        # All tensors must be on the common "inner" grid: (B, 1022, 199)
        # We lose 2 points in Space (dim=1) and 2 in Time (dim=2)

        # slice both X and T
        u_inner = u[:, 1:-1, 1:-1] 

        # slice only in X
        u_t_inner = u_t[:, 1:-1, :]  

        # slice only in T
        u_x_inner = u_x[:, :, 1:-1]  

        # slice only in T
        u_xx_inner = u_xx[:, :, 1:-1] 

        # --- 5. Calculate Residual ---
        residual = u_t_inner + u_inner * u_x_inner - self.viscosity * u_xx_inner
        return residual

    def forward(self, x_in, y_pred, y_true):
        """
        Calculates the total loss.
        x_in:   Input (t=0). Shape (B,1024,1)
        y_pred: Prediction (t=1..200). Shape (B, 1024, 200)
        y_true: Ground truth (t=1..200). Shape (B, 1024, 200 )
        """

        # 1. Data Loss (L_data)
        # This is correct. LpLoss will compare (B, 1024, 200) tensors.
        data_loss = self.data_loss_fn(y_pred, y_true)

        # 2. Physics Loss (L_physics)

        u_full_pred = torch.cat((x_in, y_pred), dim=2) # Shape: (B, 1024, 201)

        # Calculate the residual
        residual = self.cal_residual_loss(u_full_pred)

        # Physics loss is the MSE of the residual
        physics_loss = F.mse_loss(residual, torch.zeros_like(residual))

        return data_loss, physics_loss

### 2.3 Data Loss function for autoregressive

In [10]:
class DataLossOnly(nn.Module):
    def __init__(self,size_average = True):
        super(DataLossOnly,self).__init__()
        self.size_average = size_average 
        self.data_loss_fn = LpLoss(size_average)
    def forward(self,x_current,y_pred,y_target): 
        data_loss = self.data_loss_fn(y_pred,y_target) 

        return data_loss, torch.tensor(0.0).to(device)
    

## 3 Training & Evaluating process

### 3.1 Training and testing loops + printing time method

In [11]:
# TRAINING LOOP (Physics-Informed Experience Learning - PIEL)
def training_loop(model: torch.nn.Module,
                  data_loader: torch.utils.data.DataLoader,
                  phys_weight:float,
                  loss_fn : torch.nn.Module,
                  optimizer: torch.optim.Optimizer,
                  device,
                  noise_level,
                  ROLLOUT_STEPS,
                  SAMPLING_PROB):
    
    # --- INITIALIZATION ---
    model.train()
    
    # These track the *total* accumulated loss for reporting over the epoch
    total_data_loss_epoch = 0.0
    total_phys_loss_epoch = 0.0
    total_samples = 0

    # --- BATCH ITERATION ---
    for batch, (x_batch_in, y_batch_target) in enumerate(data_loader):
        x_batch_in, y_batch_target = x_batch_in.to(device), y_batch_target.to(device)
        if noise_level >0.0: 
            noise = torch.randn_like(x_batch_in)*x_batch_in.std() * noise_level 
            x_batch_in +=  noise
        
        # Current input window: Anchor the sequence to the initial history
        x_current = x_batch_in 
        loss_accumulate = 0.0

        # Check if y_batch has enough steps (safety check, 5 steps needed)
        if y_batch_target.size(2) < ROLLOUT_STEPS:
             # Skip or handle error if the dataset didn't provide enough steps
             print(f"Warning: Batch {batch} skipped. Expected {ROLLOUT_STEPS} steps, got {y_batch_target.size(2)}.")
             continue

        rollout_loss_accumulated =0.0
        # --- PIEL ROLLOUT LOOP (Accumulates Loss for BPTT) ---
        for step in range(ROLLOUT_STEPS):
            
            # 1. Forward Pass
            y_pred_step = model(x_current)

            # 2. Scheduled Sampling (PIEL)
            if torch.rand(1) < SAMPLING_PROB:
                # Teacher Forcing: Use the ground truth for the next input window
                next_input = y_batch_target[:, :,:, step*CHANNELS:(step+1)*CHANNELS]
            else:
                # Self-Supervision (PIEL): Use the model's prediction for the next input window
                next_input = y_pred_step
            
            # 3. Accumulate Loss for this step
            # Ground truth for this step is y_batch_target[:, :,:, step*2:(step+1)*2]
            data_loss_step, phys_loss_step = loss_fn(
                x_current, 
                y_pred_step, 
                y_batch_target[:, :, :,step*CHANNELS:(step+1)*CHANNELS]
            )
            
            # Sum up the weighted loss contributions
            rollout_loss_accumulated += data_loss_step + phys_weight * phys_loss_step
            
            # 4. Advance Input Window (Crucial for BPTT)
            # Drop the oldest step (t=0) and add the sampled step (t=11). 
            # The assignment is necessary to maintain the computational graph.
            x_current = torch.cat(
                (x_current[:, :, :, CHANNELS:], next_input), 
                dim=-1
            )
            
            # --- Tracking Metrics for Reporting ---
            total_data_loss_epoch += data_loss_step.item()
            total_phys_loss_epoch += phys_loss_step.item()
            total_samples += 1
            
        # --- END OF ROLLOUT LOOP ---
        
        # 5. Backward Pass and Optimization (Done once per batch for the full accumulated rollout loss)
        optimizer.zero_grad()
        rollout_loss_accumulated.backward() 
        optimizer.step()
        
    # --- SCALING FOR REPORTING (Average over all steps and batches) ---
    train_data_loss = total_data_loss_epoch / total_samples
    train_phys_loss = total_phys_loss_epoch / total_samples

    return train_data_loss, train_phys_loss

In [12]:
#Testing loop
def testing_loop(model:torch.nn.Module,
                 data_loader: torch.utils.data.DataLoader,
                 loss_fn:torch.nn.Module,
                 device):

  model.eval()
  val_data_loss = 0
  with torch.inference_mode(): # Disable gradient calculation
    for batch,(x_batch, y_batch) in enumerate(data_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        y_pred = model(x_batch)

        # We only care about the data loss for validation
        data_loss, _ = loss_fn(x_batch, y_pred, y_batch[:,:,:,0:2]) #First step

        #Accumulating
        val_data_loss += data_loss.item()


  #Scaling
  val_data_loss /= len(data_loader)

  return val_data_loss


In [13]:
def printing_time(start, end, model_name):
    print(f"It takes {(end-start):.2f} s to train {model_name}")

### 3.2 Traing & validating for epochs



In [14]:
import torch
from tqdm.auto import tqdm
from timeit import default_timer as timer

def train_and_test(lr_scheduler, model: torch.nn.Module,
                   loss_fn:torch.nn.Module, optimizer: torch.optim.Optimizer,phys_weight :float, train_loader,
                   dev_loader, epochs = 100, device = device,noise_level =0.0, rollout_steps =5, sampling_prob =0.9):
  #Setups
  model.to(device)

  results = {
      "train_data_loss":[],
      "train_phys_loss":[],
      "val_data_loss":[],
  }

  #Start timing
  start_time = timer()

  #Start
  for epoch in tqdm(range(epochs)):
    if epoch %8 == 0:
      print("--------------------------")
      print(f"Epoch: {epoch}\n--------------")
        
      sampling_prob -=0.1

    #Training
    train_data_loss, train_phys_loss = training_loop(model,train_loader, phys_weight, loss_fn, optimizer,device,noise_level,rollout_steps,sampling_prob)

    #Testing
    val_data_loss = testing_loop(model,dev_loader,loss_fn,device)

    #Decay lr based on val_loss
    lr_scheduler.step(val_data_loss)

    #Printing
    if epoch % 8 ==0:
      print(f"Train data Loss:{train_data_loss:.3f} || Train physic Loss:{train_phys_loss:.3f} || Test Loss:{val_data_loss:.3f}")

    #Restore values
    results["train_data_loss"].append(train_data_loss.item() if isinstance(train_data_loss, torch.Tensor) else train_data_loss)
    results["train_phys_loss"].append(train_phys_loss.item() if isinstance(train_phys_loss, torch.Tensor) else train_phys_loss)
    results["val_data_loss"].append(val_data_loss.item() if isinstance(val_data_loss, torch.Tensor) else val_data_loss)

  #Printing time
  print("--------------------------")
  end_time = timer()
  printing_time(start_time,end_time,type(model).__name__)

  return results


### 3.3 Evaluating model function

In [15]:
def evaluating(model: torch.nn.Module, loss_fn:torch.nn.Module,
               data_loader:torch.utils.data.DataLoader, device =device):

  model.to(device)

  test_loss =0

  #Evaluating
  model.eval()
  with torch.inference_mode():
    for x_batch, y_batch in data_loader:
      x_batch , y_batch = x_batch.to(device), y_batch.to(device)


      #Forward
      y_pred = model(x_batch)

      #Loss and acc
      loss,_= loss_fn(x_batch,y_pred,y_batch)
      test_loss += loss
  print("Finished evaluating your model")

  return {"model" : model.__class__.__name__,
          "loss":test_loss.item()/len(data_loader)}

# III. Models 1D

## 1.DATA

### 1.1 T1 data

In [16]:
#Data for model using 1time step input
data_t1 = BurgerDataset_t1(sol_tensor)

# Attribute of dataset
total_len = len(data_t1)
train_size = int(total_len * 0.6 )
dev_size = int(total_len *0.2)
test_size = total_len - train_size - dev_size
BATCH_SIZE = 32
print(f"Number of samples: {total_len}")

#Split datasets
train_dataset_t1 , dev_dataset_t1, test_dataset_t1 = random_split(data_t1, [train_size, dev_size, test_size])


# Generating loader
g = torch.Generator()
g.manual_seed(RAND_SEED)

train_loader_t1 = DataLoader(
    dataset = train_dataset_t1,
    batch_size = BATCH_SIZE,
    shuffle  = True,
    num_workers = 0,
    generator = g
)

dev_loader_t1 = DataLoader(
    dataset = dev_dataset_t1,
    batch_size = BATCH_SIZE,
    shuffle  = False,
    num_workers = 0,
    generator = g

)

test_loader_t1 = DataLoader(
    dataset = test_dataset_t1,
    batch_size = BATCH_SIZE,
    shuffle  = False,
    num_workers =0,
    generator = g
)

x_z,y_z = next(iter(train_loader_t1))
print(x_z.shape)
print(y_z.shape)
print("\nDataLoaders1t created successfully.")

NameError: name 'BurgerDataset_t1' is not defined

### 1.2 T10 data

In [17]:
import numpy as np 
from numpy import random

#Attributes of the data set
T_in = 10
T_out = 5
BATCH_SIZE = 32
WINDOW_PER_SIM_TRAIN = 20 # 20 random windows per sim
WINDOW_PER_SIM_DEV = 50  

# --- 2. Create the new DataLoaders ---
train_data_t10 = CylinderFlow_Autoregressive_Sparse(train_path, t_in=T_in, t_out=T_out, windows_per_sim=WINDOW_PER_SIM_TRAIN)
dev_data_t10 = CylinderFlow_Autoregressive_Sparse(dev_path, t_in=T_in, t_out=T_out, windows_per_sim=WINDOW_PER_SIM_DEV)
test_data_t10 = CylinderFlow_Autoregressive_Sparse(test_path, t_in=T_in, t_out=T_out, windows_per_sim=WINDOW_PER_SIM_DEV)

print(f"Total training samples (windows): {len(train_data_t10)}") # 60,000
print(f"Total dev samples (windows): {len(dev_data_t10)}")      # 25,000

g2 = torch.Generator().manual_seed(RAND_SEED)
train_loader_t10 = DataLoader(train_data_t10, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, generator=g2, pin_memory=True)
dev_loader_t10 = DataLoader(dev_data_t10, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
test_loader_t10 = DataLoader(test_data_t10, batch_size=BATCH_SIZE, shuffle=True, num_workers=0,pin_memory=True)


Total training samples (windows): 20000
Total dev samples (windows): 5000


## 4. Model 4
**(Plautaue at .. epoch after .. hour)

** Model 4 (Autoregressive, 3-in) Test Loss: 

- use data_t10 
- FNO2D
- DatalossOnly 

### 4.1 Loss function and optimizer

In [26]:
## HYPER Parameters from the paper
WIDTH =64
T_IN =10
T_OUT =1
MODE_X = 12
MODE_Y = 12

#Define model
model4 = FNO2D(in_chanels=T_IN*CHANNELS, 
                 out_chanels=T_OUT*CHANNELS, 
                 mode_x=MODE_X, 
                 mode_y=MODE_Y, 
                 width=WIDTH).to(device)

In [27]:
loss_fn4 = DataLossOnly(size_average =True)

In [28]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

#Optimizer (Adam)
LR =0.001
optimizer4 = torch.optim.Adam(model4.parameters(),lr = LR)

# We'll use 'ReduceLROnPlateau'.
# This scheduler will automatically "reduce" the learning rate
# when it detects that our validation loss has "plateaued" (stopped improving).
scheduler4 = ReduceLROnPlateau(
    optimizer4,
    mode='min',      # It will look at the validation loss, where 'min' is better
    factor=0.1,      # Reduce LR by a factor of 10 (e.g., 0.001 -> 0.0001)
    patience=5,      # Wait 5 epochs for improvement before reducing
)

### 4.2 Training and testing model 4

In [29]:
# --- HYPERPARAMETERS ---
# ROLLOUT_STEPS must match the number of output steps provided by your DataLoader's y_batch.

ROLLOUT_STEPS = 5  
SAMPLING_PROB = 0.9 
EPOCHS =16
NOISE_LEVEL =0.05
PHYS_WEIGHT =0.0

result4 = train_and_test(scheduler4, model = model4,
              loss_fn = loss_fn4, optimizer = optimizer4,phys_weight = PHYS_WEIGHT, train_loader = train_loader_t10,
              dev_loader = dev_loader_t10, epochs = EPOCHS,device = device, noise_level = NOISE_LEVEL, rollout_steps =ROLLOUT_STEPS , sampling_prob =SAMPLING_PROB)

  0%|          | 0/16 [00:00<?, ?it/s]

--------------------------
Epoch: 0
--------------
Train data Loss:0.056 || Train physic Loss:0.000 || Test Loss:0.028
--------------------------
Epoch: 8
--------------
Train data Loss:0.014 || Train physic Loss:0.000 || Test Loss:0.009
--------------------------
It takes 3783.50 s to train FNO2D


In [22]:
    print(f"Result 4: {result4}")

Result 4: {'train_data_loss': [0.05087211075901985, 0.019582114146053792, 0.01718435420244932, 0.01622707962244749, 0.013885308791100979, 0.012675398913770914, 0.014305818380713462, 0.01390168257534504, 0.013852717347741126, 0.010688792301565409, 0.003958933025971055, 0.003676559438854456, 0.0035291603688895703, 0.0034527564647048713, 0.0033684210056811573, 0.003355002436041832], 'train_phys_loss': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'val_data_loss': [0.014770651550572009, 0.011961442513308327, 0.01349966850274118, 0.008752051269291503, 0.02810782082616144, 0.013955968230440739, 0.02735861696919818, 0.013814288179633344, 0.01048888521755387, 0.009351865329133098, 0.003875018720319317, 0.0037171958306519565, 0.0036459622057866616, 0.0034536947466575416, 0.0033496209067310307, 0.003377579756164152]}


## 6. Saving and comparing models

### 6.1 Saving model

In [ ]:
import torch
import os

print("Saving initial untrained models...")

# 1. Define a path to save your models in your Drive
MODELS_PATH = Path("models")
os.makedirs(MODELS_PATH, exist_ok=True)

# 2. Define the save paths for each model
MODEL4_SAVE_PATH = MODELS_PATH / "model4_initial_state.pth"


# 3. Save the models' state dictionaries
# .state_dict() saves only the learnable parameters (weights and biases)
try:
    torch.save(model4.state_dict(), MODEL4_SAVE_PATH)

    print(f"Successfully saved model4 to: {MODEL4_SAVE_PATH}")

except Exception as e:
    print(f"Error saving models: {e}")

# To load them back later (for inference or to resume training), you would:
# 1. Re-create the model instance: model0 = FNO1D(WIDTH, K_MAX, x_grid_tensor).to(device)
# 2. Load the weights: model0.load_state_dict(torch.load(MODEL0_SAVE_PATH))

### 6.2 Compare models

In [30]:
import torch
import torch.nn.functional as F
import numpy as np

# This must match your model and dataset
GRID_SIZE = 64
CHANNELS = 2 # vx, vy
T_IN = 10    # 10 steps in

def evaluate_autoregressive_rollout(model, dataset, loss_fn,t_in, device):
    """
    Performs a full autoregressive rollout on the 2D grid dataset
    by loading each sample file.
    
    Args:
        model: Your trained 2D FNO model
        dataset: The CylinderFlowGridDataset object (e.g., dev_dataset)
        loss_fn: An instance of your 2D LpLoss (NOT DataLossOnly)
        device: Your "cuda" or "cpu" device
    """
    model.eval()
    total_rollout_loss = 0.0
    
    with torch.no_grad():
        # Loop through each sample file in the dataset
        for i in tqdm(range(len(dataset.file_paths)), desc="Evaluating Rollout"):
            
            # 1. Load the full 600-step trajectory for one sample
            data = torch.load(dataset.file_paths[i])
            # Shape: (600, 64, 64, 2)
            velocity_traj = data['velocity'].to(device).to(torch.float32)

            # --- 2. Create the first input window ---
            # Get t=0...9
            x_window_t = velocity_traj[:t_in] # (10, 64, 64, 2)
            
            # Reshape to (1, 64, 64, 20) for the model
            current_window = x_window_t.permute(1, 2, 0, 3) 
            current_window = current_window.reshape(GRID_SIZE, GRID_SIZE, t_in * CHANNELS)
            current_window = current_window.unsqueeze(0) # Add batch dim

            # --- 3. Create the ground truth for comparison ---
            # Get t=10...599
            y_true_all_steps = velocity_traj[t_in:] # (590, 64, 64, 2)
            
            # Reshape to (1, 64, 64, 1180)
            y_true_rollout = y_true_all_steps.permute(1, 2, 0, 3)
            y_true_rollout = y_true_rollout.reshape(GRID_SIZE, GRID_SIZE, -1)
            y_true_rollout = y_true_rollout.unsqueeze(0)

            
            predictions = [] # List to store all 590 predictions
            n_steps = y_true_all_steps.shape[0] # 590 steps
            
            # --- 4. Start the rollout loop ---
            for _ in range(n_steps):
                # model input: (1, 64, 64, 20)
                # model output: (1, 64, 64, 2)
                y_pred_step = model(current_window)
                
                if torch.any(torch.isinf(y_pred_step)) or torch.any(torch.isnan(y_pred_step)):
                    print(f"!!! Rollout became unstable at step {i} !!!")
                    break 
                
                predictions.append(y_pred_step)
                
                # --- 5. Feed back in ---
                # Drop oldest 2 channels, add new 2 channels
                current_window = torch.cat(
                    (current_window[:, :, :, CHANNELS:], y_pred_step), 
                    dim=3 # Concatenate on the channel dimension
                )
            
            # --- 6. Stack all predictions ---
            # y_pred_full_rollout shape: (1, 64, 64, 1180)
            if len(predictions) != n_steps: # Handle unstable rollout
                print("Rollout failed, skipping this sample.")
                continue
                
            y_pred_full_rollout = torch.cat(predictions, dim=3)
            
            # --- 7. Calculate final loss ---
            # Use the simple LpLoss, not the DataLossOnly wrapper
            rollout_loss = loss_fn(x_window_t,y_pred_full_rollout, y_true_rollout)
            total_rollout_loss += rollout_loss[0]

    # Return the average loss over all test samples
    return total_rollout_loss / len(dataset.file_paths)

print("evaluate_autoregressive_rollout (2D version) function defined.")

evaluate_autoregressive_rollout (2D version) function defined.


In [31]:
# (Run this in a new cell after training model2)

print("Starting FAIR comparison rollout for Model 2 and 3...")

# Use test_loader_t1 (from cell 17) for a fair test
# Use loss_fn0 (from cell 55) for a fair data-only loss

rollout_loss_m4 = evaluate_autoregressive_rollout(
    model=model4, 
    dataset=test_data_t10, 
    loss_fn=loss_fn4,           
    device=device,
    t_in=10# <-- This correctly matches your model2
)

print("\n--- Fair Comparison Test Results ---")
print(f"Model 0 (Direct, Data-Only) Test Loss:     0.0617") # From cell 105
print(f"Model 1 (Direct, PINO, w=0.1) Test Loss:   0.063") # From cell 111
print(f"Model 4 (Autoregressive, 3-in) Test Loss: {rollout_loss_m4:.4f}")

Starting FAIR comparison rollout for Model 2 and 3...


Evaluating Rollout:   0%|          | 0/100 [00:00<?, ?it/s]


--- Fair Comparison Test Results ---
Model 0 (Direct, Data-Only) Test Loss:     0.0617
Model 1 (Direct, PINO, w=0.1) Test Loss:   0.063
Model 4 (Autoregressive, 3-in) Test Loss: 0.3364


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from results import result1, result0

# --- IMPORTANT ---
# You must copy the dictionary outputs from your notebook cells
# (cell 50 for model0 and cell 67 for model1) and paste them here.
# I have used your completed output from cell 50 as an example.
# You will need to replace results_1 with your full 30-epoch output.

# --- Create Epoch Array ---
# Assumes both models were trained for the same number of epochs
result0 = {'train_data_loss': [0.4110739827156067, 0.23925882577896118, 0.20458190143108368, 0.19567212462425232, 0.17208527028560638, 0.16243018209934235, 0.16161102056503296, 0.1497381031513214, 0.1424730122089386, 0.13952957093715668, 0.13167569041252136, 0.1314629167318344, 0.13347485661506653, 0.1252565085887909, 0.12282412499189377, 0.12299676984548569, 0.11490682512521744, 0.11774254590272903, 0.10972394794225693, 0.10895457118749619, 0.11255241930484772, 0.10872527211904526, 0.10500839352607727, 0.10531488806009293, 0.10197656601667404, 0.10225849598646164, 0.10453744977712631, 0.10029015690088272, 0.09715524315834045, 0.10300806164741516, 0.09627469629049301, 0.09561239928007126, 0.09352393448352814, 0.09420380741357803, 0.09108910709619522, 0.08958891034126282, 0.08952014893293381, 0.0945991724729538, 0.08834867179393768, 0.08973327279090881, 0.08752944320440292, 0.08816472440958023, 0.08264376223087311, 0.08628880232572556, 0.08571340143680573, 0.08524730801582336, 0.08302949368953705, 0.08613189309835434, 0.08262985199689865, 0.0860193744301796, 0.08372187614440918, 0.08175082504749298, 0.0784018486738205, 0.07959341257810593, 0.0778963565826416, 0.0784391239285469, 0.07724633812904358, 0.07945655286312103, 0.07577656954526901, 0.08160007745027542, 0.08002155274152756, 0.07753711193799973, 0.07859975844621658, 0.0764986202120781, 0.07406435161828995, 0.0723976269364357, 0.0739876925945282, 0.07461126893758774, 0.07415719330310822, 0.07250193506479263, 0.07158296555280685, 0.0725533589720726, 0.07088251411914825, 0.0713513195514679, 0.07630784064531326, 0.07213081419467926, 0.06057237461209297, 0.05909671634435654, 0.05885971337556839, 0.058606330305337906, 0.05847145989537239, 0.058534953743219376, 0.05826769769191742, 0.05820904299616814, 0.05816391482949257, 0.05802098661661148, 0.058009110391139984, 0.057899314910173416, 0.0577441044151783, 0.05807175859808922, 0.057901881635189056, 0.05766398832201958, 0.05750136449933052, 0.05765574425458908, 0.05767909809947014, 0.05718502774834633, 0.0571594312787056, 0.057440076023340225, 0.057252444326877594, 0.05690488591790199, 0.05690895393490791, 0.05691118910908699, 0.056899115443229675, 0.05672742426395416, 0.05663994699716568, 0.05657578259706497, 0.05658656731247902, 0.05656237527728081, 0.05650690570473671, 0.05641086399555206, 0.056626658886671066, 0.05617628991603851, 0.05610703304409981, 0.05624675750732422, 0.0564168281853199, 0.05596563592553139, 0.05595419928431511, 0.055922769010066986, 0.05574888736009598, 0.05567246302962303, 0.05585700646042824, 0.055710289627313614, 0.05597971752285957, 0.05580592527985573, 0.05584094300866127, 0.05574943125247955, 0.05546464025974274, 0.055211909115314484, 0.05526223033666611, 0.05522475764155388, 0.05564416944980621, 0.05541204661130905, 0.05530177429318428, 0.05499405786395073, 0.05497822165489197, 0.055076297372579575, 0.05496148020029068, 0.0547984354197979, 0.05514082685112953, 0.055032216012477875, 0.05497255548834801, 0.05481015145778656, 0.05463377758860588, 0.054746970534324646, 0.05454118177294731, 0.05435951054096222, 0.05476135388016701, 0.054547540843486786, 0.05460606887936592, 0.05433722585439682], 'train_phys_loss': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'val_data_loss': [0.2572142621354451, 0.21145666733620658, 0.19115776675088064, 0.1802690809681302, 0.17118003964424133, 0.15824853404173775, 0.17612133447139983, 0.14190959067098677, 0.13687605919345977, 0.1335370416442553, 0.13127026198402283, 0.1297357457261237, 0.12929621764591762, 0.12160444259643555, 0.15860963624621194, 0.12193896668770957, 0.13977357768823231, 0.11167937008634446, 0.11051414717757513, 0.11135667432395238, 0.11710662323804129, 0.10518188218748759, 0.10574372337451057, 0.11249114868659822, 0.10047837618797545, 0.10142433477772607, 0.09888798826270634, 0.09998914327413316, 0.10522396519543632, 0.10073239829332109, 0.09524016141418427, 0.09673324570296303, 0.0917700073785252, 0.09177173449406548, 0.09537116840245231, 0.0898829839295811, 0.09122747289282936, 0.09619156460440348, 0.09295563683623359, 0.09442581981420517, 0.0865695761546256, 0.08646621212126716, 0.08684119321997204, 0.089255215156646, 0.0872151656519799, 0.08562316390730086, 0.0906154742789647, 0.08481654382887341, 0.08532685396217164, 0.09498826461651969, 0.08751835993358068, 0.08428332919166201, 0.10068383375330577, 0.08187451710303624, 0.08577221821224879, 0.07869435054442239, 0.08390380430316167, 0.08048265462829954, 0.07998732867695037, 0.07940924309548877, 0.08420824247693258, 0.07788392381062584, 0.08739961293481645, 0.07972035036673622, 0.07721601213727679, 0.07708905519001068, 0.08067990259991752, 0.07722300066361351, 0.09537790822131294, 0.07313832954045325, 0.07719190785336116, 0.0766766376438595, 0.07478425805530851, 0.0733860337308475, 0.07394332036612526, 0.07324050420096942, 0.06501247840268272, 0.06459305080629531, 0.06465581023976916, 0.06449638975281564, 0.06492461772665145, 0.06398267458592143, 0.0644972435538731, 0.06389726597874884, 0.06378511274381289, 0.06382066389871022, 0.06389406625004042, 0.06369595667199483, 0.06350627009357725, 0.0635057757534678, 0.06327962041610763, 0.06329736746256313, 0.06380144559911319, 0.06318371796182223, 0.06303894472500635, 0.06305838967599565, 0.06306172854134015, 0.06316362587468964, 0.06280702772358107, 0.06297555837839369, 0.06427314351238901, 0.06272708405814474, 0.06253687392861124, 0.06268517643449799, 0.062455795646186855, 0.06254122944341765, 0.06238306195489944, 0.06241820912275996, 0.062523636493891, 0.06233104856477843, 0.0623762799752137, 0.06204700215704857, 0.06217261888678112, 0.06274812715867209, 0.0618926150694726, 0.06204395309563667, 0.06187254799500344, 0.061938941537860844, 0.06266835385135242, 0.06309285830883753, 0.06154843940148278, 0.06175081975876339, 0.06152673846199399, 0.06216449757653569, 0.06183923437954888, 0.06193630456451386, 0.06164784404256987, 0.06338022170322281, 0.06135745565333064, 0.06773435566869992, 0.06207913467808375, 0.0613168551926575, 0.061097964880958436, 0.06094158074212453, 0.06099840265417856, 0.06099581180347337, 0.06111503788639629, 0.06091841511310093, 0.06254472116392756, 0.06069481703970167, 0.060798205198749664, 0.0615965946917496, 0.060730867383499, 0.06066012004065135, 0.06074823173029082, 0.0605203460842844, 0.060685241033160496, 0.06406244617842492, 0.06041179910775215, 0.06047245848273474]}

epochs = range(len(result0['train_data_loss']))


# --- Create Plots ---
plt.figure(figsize=(18, 5))

# Plot 1: Training Data Loss Comparison
plt.subplot(1, 3, 1)
plt.plot(epochs, result0['train_data_loss'], label='Model 0 (Data Only)', color='blue')
plt.plot(epochs, result1['train_data_loss'], label='Model 1 (PINO)', color='red')
plt.title('Training Data Loss')
plt.xlabel('Epochs')
plt.ylabel('Relative L2 Loss (L_data)')
plt.legend()
plt.grid(True, linestyle=':')
plt.yscale('log') # Use log scale if losses are very different

# Plot 2: Validation Data Loss Comparison
plt.subplot(1, 3, 2)
plt.plot(epochs, result0['val_data_loss'], label='Model 0 (Data Only)', color='blue')
plt.plot(epochs, result1['val_data_loss'], label='Model 1 (PINO)', color='red')
plt.title('Validation Data Loss')
plt.xlabel('Epochs')
plt.ylabel('Relative L2 Loss (L_data)')
plt.legend()
plt.grid(True, linestyle=':')
plt.yscale('log')

# Plot 3: Model 1 (PINO) Loss Breakdown
plt.subplot(1, 3, 3)
plt.plot(epochs, result1['train_data_loss'], label='Data Loss', color='red')
plt.plot(epochs, result1['train_phys_loss'], label='Physics Loss', color='green')
plt.title('Model 1 (PINO) Loss Components')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle=':')
plt.yscale('log') # Log scale is essential here since physics loss is large



plt.tight_layout()
plt.show()